In [122]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

train = pd.read_csv('~/Downloads/train.csv')
test = pd.read_csv('~/Downloads/test.csv')

#Verified all columns are available, no need impute
#na_cols = train.columns[train.isna().any()].tolist()
y_train = train['count']
# 'temp' and 'atemp' high correlated - drop one of them
# Test dataset not contain 'casual' and 'registered', drop
train = train.drop(['count', 'casual', 'registered', 'atemp', 'datetime'], axis=1)
test = test.drop(['atemp', 'datetime'], axis=1)

print(train.shape)
print(test.shape)
#train[:24*10].plot(x='datetime', y='count')

(8600, 7)
(2286, 7)


In [123]:
# One hot encoding - categorical variables
def one_hot_encoding(data):
    dummy_fields = ['season', 'weather']
    for each in dummy_fields:
        dummies = pd.get_dummies(data[each], prefix=each, drop_first=False)
        data = pd.concat([data, dummies], axis=1)

    # 'temp' and 'atemp' high correlated - drop one of them
    fields_to_drop = ['season', 'weather']
    one_hot_data = data.drop(fields_to_drop, axis=1)
    return one_hot_data
    
# Test dataset missing some weather 4, so combine together for one_hot and then split
concat_data = pd.concat([train, test])
data = one_hot_encoding(concat_data)
train_data = data[:train.shape[0]]
test_data = data[train.shape[0]:]
print(data.shape)
print(train_data.shape)
print(test_data.shape)
#train_data.head(2)
test_data.head(2)

(10886, 13)
(8600, 13)
(2286, 13)


,holiday,workingday,temp,humidity,windspeed,season_1,season_2,season_3,season_4,weather_1,weather_2,weather_3,weather_4
0,0,0,10.66,56,0.0000,1,0,0,0,1,0,0,0
1,0,0,10.66,56,8.9981,1,0,0,0,1,0,0,0


In [124]:
# standardize
#from sklearn.preprocessing import StandardScaler

#scaler_x = StandardScaler()
#x_train = scaler_x.fit_transform(train_data)
#x_test = scaler_x.transform(test_data)

x_train = train_data
x_test = test_data
print(x_train.shape)
print(x_test.shape)
#x_train.head(1)
#list(y_train)

(8600, 13)
(2286, 13)


In [130]:
# Using linear regression might obtain negative prediction values
# Instead, using log-price, generalized linear models (GLMs), poisson
#from sklearn.linear_model import LinearRegression
#from sklearn.linear_model import SGDRegressor
#from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
#from sklearn.ensemble import RandomForestRegressor

#model = LinearRegression()
#model = SGDRegressor()
#model = RandomForestRegressor()
#model = GradientBoostingRegressor()
model = ExtraTreesRegressor()
model.fit(x_train, y_train)

/Users/mingxie/.pyenv/versions/3.7.0/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
          oob_score=False, random_state=None, verbose=0, warm_start=False)

In [131]:
pred = model.predict(x_test)

In [132]:
#import numpy as np
#pred = np.ndarray.round(pred)
#pred = np.floor(pred)
#pred = np.ceil(pred)
#pred = np.around(pred)
submission_file = pd.read_csv('~/Downloads/submission_sample.csv')
submission = pd.DataFrame({'datetime': submission_file['datetime'], 'count': pred})
submission.to_csv('~/Downloads/extra_submission.csv', index=False)